In [1]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'AffectNet_train_set/seven/'
validation = ALL_DATA_DIR + 'AffectNet_val_set/seven/'

In [2]:
batch_size = 32 
img_height = 224
img_width = 224 

In [3]:
def preprocess(image, label):
    final_image = keras.applications.xception.preprocess_input(image)
    return final_image, label

In [4]:
validation_s = tf.keras.utils.image_dataset_from_directory(validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)
training_s = tf.keras.utils.image_dataset_from_directory(training, labels= 'inferred',
            image_size=(img_height, img_width), batch_size=None, seed=42)

Found 3500 files belonging to 7 classes.
Found 283901 files belonging to 7 classes.


In [5]:
validation_ds = validation_s.map(preprocess).batch(batch_size).prefetch(1)
training_ds = training_s.map(preprocess).batch(batch_size).prefetch(1)

In [6]:
base_model = keras.applications.Xception(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_ds, validation_data=validation_ds, epochs=5)

Epoch 1/5
8872/8872 [==============================] - 1250s 140ms/step - loss: 1.0863 - accuracy: 0.6107 - val_loss: 1.9899 - val_accuracy: 0.3171
Epoch 2/5
8872/8872 [==============================] - 1271s 143ms/step - loss: 1.0377 - accuracy: 0.6269 - val_loss: 2.0065 - val_accuracy: 0.3200
Epoch 3/5
8872/8872 [==============================] - 1270s 143ms/step - loss: 1.0261 - accuracy: 0.6311 - val_loss: 2.0513 - val_accuracy: 0.3243
Epoch 4/5
8872/8872 [==============================] - 1270s 143ms/step - loss: 1.0209 - accuracy: 0.6324 - val_loss: 2.1394 - val_accuracy: 0.3203
Epoch 5/5
8872/8872 [==============================] - 1270s 143ms/step - loss: 1.0170 - accuracy: 0.6334 - val_loss: 2.0202 - val_accuracy: 0.3200


In [11]:
for layer in base_model.layers:
    layer.trainable = True

In [12]:
opt = keras.optimizers.Adam(learning_rate=1e-4) #with unfreezing, lessen the learning rate (0.0001)

In [13]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True) 
#why? if there is no loss difference between epochs=> stop the fitting 

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [15]:
history = model.fit(training_ds, validation_data=validation_ds, epochs=10, callbacks=[es])
model.save('0920_Affect_Xception_eval.h5')

Epoch 1/10
8872/8872 [==============================] - 5516s 621ms/step - loss: 0.6699 - accuracy: 0.7565 - val_loss: 1.2189 - val_accuracy: 0.5631
Epoch 2/10
8872/8872 [==============================] - 5452s 614ms/step - loss: 0.5461 - accuracy: 0.7983 - val_loss: 1.2200 - val_accuracy: 0.5857
Epoch 3/10
8872/8872 [==============================] - 5492s 619ms/step - loss: 0.4211 - accuracy: 0.8421 - val_loss: 1.5737 - val_accuracy: 0.5740
Epoch 4/10
8872/8872 [==============================] - 5481s 618ms/step - loss: 0.2877 - accuracy: 0.8915 - val_loss: 1.9973 - val_accuracy: 0.5751
Epoch 5/10
8872/8872 [==============================] - 5511s 621ms/step - loss: 0.2076 - accuracy: 0.9219 - val_loss: 2.2307 - val_accuracy: 0.5514
Epoch 6/10
8872/8872 [==============================] - 5506s 621ms/step - loss: 0.1669 - accuracy: 0.9376 - val_loss: 2.3605 - val_accuracy: 0.5671


In [16]:
model.evaluate(validation_ds)

110/110 [==============================] - 16s 139ms/step - loss: 1.2189 - accuracy: 0.5631


[1.2189241647720337, 0.5631428360939026]

In [17]:
model.evaluate(training_ds)

8872/8872 [==============================] - 1256s 142ms/step - loss: 0.5464 - accuracy: 0.7989


[0.5463612079620361, 0.7988594770431519]